In [ ]:
from netCDF4 import Dataset
from matplotlib import pyplot as plt
import numpy as np
import datetime
from scipy.interpolate import griddata
from mpl_toolkits.basemap import Basemap
import os

In [ ]:
start_time_era5 = datetime.datetime(1900, 1, 1, 0, 0, 0)
lon_min_era5, lon_max_era5, lat_min_era5, lat_max_era5 = 50, 110, 70, 83

In [ ]:
file = '/mnt/hippocamp/DATA/ERA5/w10/era5_uv10m_2020-09.nc'
data = Dataset(file, 'r')

longitude = np.array(data.variables['longitude'][lon_min_era5*4:lon_max_era5*4+1])
latitude = np.array(data.variables['latitude'][(90-lat_max_era5)*4:(90-lat_min_era5)*4+1])
lon_grid, lat_grid = np.meshgrid(longitude, latitude)

In [ ]:
def drawing(u, v, t=None, lon = lon_grid, lat = lat_grid, save=False):
    fig = plt.figure(figsize=(12, 12), dpi=300)
    
    m = Basemap(width=600000, height=700000,
                resolution='l', projection='aea',
                lat_1=60, lat_2=65, lon_0=80, lat_0=74)
    m.drawcoastlines()
    m.fillcontinents(color='grey', lake_color='white')
    m.drawparallels(np.arange(-80., 90., 2.), labels=[False, True, True, False])
    m.drawmeridians(np.arange(-180., 180., 5.), labels=[True, True, False, True])
    m.drawmapboundary(fill_color='white')
    
    # Преобразование координат для карты
    x, y = m(lon, lat)
    
    # Вычисление скорости ветра
    speed = np.sqrt(u**2 + v**2)
    
    # Отображение скорости ветра
    speed_plot = m.pcolormesh(x, y, speed, shading='gouraud', cmap='jet', vmin=0, vmax=10)
    cbar=plt.colorbar(speed_plot, label='Wind speed (m/s)')
    # cbar.set_ticks([0,2,4,6,8,10,12])
    
    # Отображение направления ветра
    m.quiver(x[::4, ::4], y[::4, ::4], u[::4, ::4], v[::4, ::4], scale=30, scale_units='inches')
    # plt.title(f'Wind speed {t.date()}')

    # Верхняя сторона (широта = 75, долготы от 75 до 85)
    top_lon = np.linspace(75, 85, 100)
    top_lat = np.full_like(top_lon, 75)

    # Нижняя сторона (широта = 72)
    bottom_lon = np.linspace(75, 85, 100)
    bottom_lat = np.full_like(bottom_lon, 72)

    # Левая сторона (долгота = 75)
    left_lat = np.linspace(72, 75, 100)
    left_lon = np.full_like(left_lat, 75)

    # Правая сторона (долгота = 85)
    right_lat = np.linspace(72, 75, 100)
    right_lon = np.full_like(right_lat, 85)

    # Отображение сторон
    m.plot(*m(top_lon, top_lat), color='blue', linewidth=2, linestyle='-')   # верх
    m.plot(*m(bottom_lon, bottom_lat), color='blue', linewidth=2, linestyle='-')  # низ
    m.plot(*m(left_lon, left_lat), color='blue', linewidth=2, linestyle='-')  # левая
    m.plot(*m(right_lon, right_lat), color='blue', linewidth=2, linestyle='-')  # правая
    
    plt.show()
    plt.close('all')

In [ ]:
def drawing_sst(sst, t=None, lon = lon_grid, lat = lat_grid, save=False):
    fig = plt.figure(figsize=(12, 12), dpi=300)
    
    m = Basemap(width=600000, height=700000,
                resolution='l', projection='aea',
                lat_1=60, lat_2=65, lon_0=80, lat_0=74)
    m.drawcoastlines()
    m.fillcontinents(color='grey', lake_color='white')
    m.drawparallels(np.arange(-80., 90., 2.), labels=[False, True, True, False])
    m.drawmeridians(np.arange(-180., 180., 5.), labels=[True, True, False, True])
    m.drawmapboundary(fill_color='white')
    
    # Преобразование координат для карты
    x, y = m(lon, lat)
    
    
    # Отображение скорости ветра
    sst_plot = m.pcolormesh(x, y, sst, shading='gouraud', cmap='jet', vmin=0)
    cbar=plt.colorbar(sst_plot, label='Temperature')
    # cbar.set_ticks([0,2,4,6,8,10,12])
    
    # plt.title(f'Wind speed {t.date()}')

    # Верхняя сторона (широта = 75, долготы от 75 до 85)
    top_lon = np.linspace(75, 85, 100)
    top_lat = np.full_like(top_lon, 75)

    # Нижняя сторона (широта = 72)
    bottom_lon = np.linspace(75, 85, 100)
    bottom_lat = np.full_like(bottom_lon, 72)

    # Левая сторона (долгота = 75)
    left_lat = np.linspace(72, 75, 100)
    left_lon = np.full_like(left_lat, 75)

    # Правая сторона (долгота = 85)
    right_lat = np.linspace(72, 75, 100)
    right_lon = np.full_like(right_lat, 85)

    # Отображение сторон
    m.plot(*m(top_lon, top_lat), color='blue', linewidth=2, linestyle='-')   # верх
    m.plot(*m(bottom_lon, bottom_lat), color='blue', linewidth=2, linestyle='-')  # низ
    m.plot(*m(left_lon, left_lat), color='blue', linewidth=2, linestyle='-')  # левая
    m.plot(*m(right_lon, right_lat), color='blue', linewidth=2, linestyle='-')  # правая
    
    plt.show()
    plt.close('all')

In [ ]:
year = 2015
month = 7
file = f'/mnt/hippocamp/DATA/ERA5/w10/era5_uv10m_{year}-{month:02}.nc'
data = Dataset(file, 'r')

In [ ]:
u10 = np.asarray(data.variables['u10'][:,(90-lat_max_era5)*4:(90-lat_min_era5)*4+1,lon_min_era5*4:lon_max_era5*4+1])

In [ ]:
v10 = np.asarray(data.variables['v10'][:,(90-lat_max_era5)*4:(90-lat_min_era5)*4+1,lon_min_era5*4:lon_max_era5*4+1])

In [ ]:
u10.shape, v10.shape

In [ ]:
year = 2015
month = 8
file = f'/mnt/hippocamp/DATA/ERA5/w10/era5_uv10m_{year}-{month:02}.nc'
data = Dataset(file, 'r')

In [ ]:
u10_ = np.asarray(data.variables['u10'][:,(90-lat_max_era5)*4:(90-lat_min_era5)*4+1,lon_min_era5*4:lon_max_era5*4+1])

In [ ]:
v10_ = np.asarray(data.variables['v10'][:,(90-lat_max_era5)*4:(90-lat_min_era5)*4+1,lon_min_era5*4:lon_max_era5*4+1])

In [ ]:
u10_.shape, v10_.shape

In [ ]:
u10 = np.concatenate((u10, u10_))
v10 = np.concatenate((v10, v10_))
u10.shape, v10.shape

In [ ]:
list_u10 = np.split(u10, u10.shape[0]/24, axis=0)
array_u10 = np.array([np.mean(a, axis=0) for a in list_u10])

list_v10 = np.split(v10, v10.shape[0]/24, axis=0)
array_v10 = np.array([np.mean(a, axis=0) for a in list_v10])

In [ ]:
array_u10.shape

In [ ]:
border_north = 75
border_south = 72
border_east = 85
border_west = 75

In [ ]:
u10_upd = array_u10[:,(lat_max_era5-border_north)*4:(lat_max_era5-border_south)*4+1,
                    (border_west-lon_min_era5)*4:(border_east-lon_max_era5)*4]
v10_upd = array_v10[:,(lat_max_era5-border_north)*4:(lat_max_era5-border_south)*4+1,
                    (border_west-lon_min_era5)*4:(border_east-lon_max_era5)*4]
u10_upd.shape, v10_upd.shape

In [ ]:
u10_upd_mean = np.mean(u10_upd, axis=(1, 2))
v10_upd_mean = np.mean(v10_upd, axis=(1, 2))
u10_upd_mean.shape, v10_upd_mean.shape

In [ ]:
V10 = np.asarray([np.sqrt(u**2 + v**2) for (u,v) in zip(u10_upd_mean, v10_upd_mean)])

In [ ]:
u10_upd_mean[24:45]

In [ ]:
v10_upd_mean[24:45]

In [ ]:
V10[24:45]

In [ ]:
u = u10_upd_mean[24:45]
v = v10_upd_mean[24:45]

x = np.arange(len(u))
y = np.zeros_like(x)

plt.figure(figsize=(8, 2), dpi=300)
plt.quiver(x, y, u, v, angles='xy', scale_units='xy', scale=1, color='blue')
plt.xlim(-8, len(u))
plt.ylim(-7, 1)
plt.xticks([i for i in range(21)])
plt.gca().set_aspect('equal')
plt.grid(True)
plt.title('Скорость и направление ветра')
plt.xlabel('Дата')
plt.ylabel('')
plt.show()

In [ ]:
u10_mean = np.mean(array_u10[31:38,:,:], axis=0)
v10_mean = np.mean(array_v10[31:38,:,:], axis=0)
u10_mean.shape, v10_mean.shape

In [ ]:
drawing(u10_mean, v10_mean)

In [ ]:
year = 2015
month = 7
file = f'/mnt/hippocamp/DATA/ERA5/sst/era5_sst_{year}-{month:02}.nc'
data = Dataset(file, 'r')

In [ ]:
sst = np.asarray(data.variables['sst'][:,(90-lat_max_era5)*4:(90-lat_min_era5)*4+1,lon_min_era5*4:lon_max_era5*4+1])

In [ ]:
year = 2015
month = 8
file = f'/mnt/hippocamp/DATA/ERA5/sst/era5_sst_{year}-{month:02}.nc'
data = Dataset(file, 'r')

In [ ]:
sst_ = np.asarray(data.variables['sst'][:,(90-lat_max_era5)*4:(90-lat_min_era5)*4+1,lon_min_era5*4:lon_max_era5*4+1])

In [ ]:
sst = np.concatenate((sst, sst_))

In [ ]:
list_sst = np.split(sst, sst.shape[0]/24, axis=0)
array_sst = np.array([np.mean(a, axis=0) for a in list_sst])
array_sst[array_sst < 0] = np.nan
array_sst -= 273
array_sst.shape


In [ ]:
sst_upd = array_sst[:,(lat_max_era5-border_north)*4:(lat_max_era5-border_south)*4+1,
                    (border_west-lon_min_era5)*4:(border_east-lon_max_era5)*4]
sst_upd.shape

In [ ]:
sst_upd_mean = np.nanmean(sst_upd, axis=(1, 2))

In [ ]:
sst_upd_mean

In [ ]:
plt.figure(figsize=(8, 4), dpi=300)
plt.plot([i for i in range(38)], sst_upd_mean[24:])
# plt.xlim(-8, len(u))
# plt.ylim(-7, 1)
plt.xticks([i for i in range(38)])
# plt.gca().set_aspect('equal')
plt.grid(True)
plt.title('SST')
plt.xlabel('Дата')
plt.ylabel('')
plt.show()

In [ ]:
sst_mean = np.mean(array_sst[38:45,:,:], axis=0)
sst_mean.shape

In [ ]:
drawing_sst(sst_mean)

In [ ]:
year = 2015
month = 7
file = f'/mnt/hippocamp/DATA/ERA5/t2/era5_t2_{year}-{month:02}.nc'
data = Dataset(file, 'r')

In [ ]:
t2 = np.asarray(data.variables['t2m'][:,(90-lat_max_era5)*4:(90-lat_min_era5)*4+1,lon_min_era5*4:lon_max_era5*4+1])

In [ ]:
year = 2015
month = 8
file = f'/mnt/hippocamp/DATA/ERA5/t2/era5_t2_{year}-{month:02}.nc'
data = Dataset(file, 'r')

In [ ]:
t2_ = np.asarray(data.variables['t2m'][:,(90-lat_max_era5)*4:(90-lat_min_era5)*4+1,lon_min_era5*4:lon_max_era5*4+1])

In [ ]:
t2 = np.concatenate((t2, t2_))

In [ ]:
list_t2 = np.split(t2, t2.shape[0]/24, axis=0)
array_t2 = np.array([np.mean(a, axis=0) for a in list_t2])
array_t2 -= 273
array_t2.shape

In [ ]:
t2_upd = array_t2[:,(lat_max_era5-border_north)*4:(lat_max_era5-border_south)*4+1,
                  (border_west-lon_min_era5)*4:(border_east-lon_max_era5)*4]
t2_upd.shape

In [ ]:
sst_upd.shape

In [ ]:
t2_upd_ = np.where(np.isnan(sst_upd), np.nan, t2_upd)

In [ ]:
t2_upd_mean = np.nanmean(t2_upd_, axis=(1, 2))

In [ ]:
t2_upd_mean

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot([i for i in range(21)], t2_upd_mean[24:45])
# plt.xlim(-8, len(u))
# plt.ylim(-7, 1)
plt.xticks([i for i in range(21)])
# plt.gca().set_aspect('equal')
plt.grid(True)
plt.title('T2')
plt.xlabel('Дата')
plt.ylabel('')
plt.show()

In [ ]:
t2_mean = np.mean(array_t2[38:45,:,:], axis=0)
t2_mean.shape

In [ ]:
drawing_sst(t2_mean)